# Quantitative Methods in Finance - Python Coursework  - Ar Rafiul Islam: 

**Coursework answers by: Ar Rafiul Islam.**

The `ipynb` file includes comments and explanations for the exercises to help the reader understand my way of thinking and understanding while writing the lines of code.

There will also be some **"Notes"** that will explain why I do things in a certain way or why sometimes running the code might give something not expected or general explanations.

----

# Introduction:

## 1. Monte Carlo:

**The Monte Carlo Method** is a mathematical technique used to estimate the possible outcomes of an uncertain event. This scheme can be applied to different fields. In this coursework we will be analysing it in the area of option pricing. 

Specifically, **Monte Carlo Method**, in regards to option pricing, involves the following steps:

1. Simulating a stock price over a defined period. It requires generating a new stock price in equally spaced time intervals over that specific timeframe. An example would be generating a new stock price every day over a year

2. Calculate the payoff and the present value of the function based on the specific parameters such as interest rate, volatility, strike price, and specific payoff function for the option

3. Then, simulate this for multiple stock realisations. The higher the number of realisations the more accurate the results will be

4. Finally, compute an average of all the discounted prices over all the realisations to find the estimated price of the option

In simple terms, the **Monte Carlo** method involves assigning multiple values to a variable that involves uncertainty (in our case how the stock price will move) and then averaging the results to get an estimate of the particular financial product.

**Note**: Although increasing the number of realisations and the time period will improve accuracy levels, it will reduce the performance in generating results.

## 2. Exotic Options:

Exotic options are a specific type of options that are traded over-the-counter (OTC) rather than exchanges due to their properties. Usually, financial products traded OTC tend to be tailored to the needs of the individual/financial institution.  

Some of the main features of Exotic options are:
1. Time dependency
2. Cashflows
3. Path dependence (Strong and Weak dependence)
4. Dimensionality (number of independent variables)

One important feature to keep in mind is path dependency. Differently from the "standard" European options where the payoff is determined only by the price of the underlying asset at maturity, with Exotic options the "path"/trajectory of the underlying asset has a big implication on its final price. We will see this more in detail in the following examples.

## 3. General Plan:

In this report I will do the following:

Introduce Asian Fixed Strike Options and price them using different parameters. Then, I will show the impact of the change of different parameters on the price of the option and compare the results.

Then, I will introduce Asian Floating Options and price them in a similar way. I will also compare how the prices for calls and puts change between the different types of strikes using a common simulation of stock prices. As an additional part, I will briefly show how the price of the Floating option changes by changing its parameters in a similar way to Fixed options.  

After, I will use Geometric averaging to price both floating and fixed strike options and compare them with Arithmetic averaging.

Finally, I will create a Supershare option, estimate some options based on different strike values, and then explain the differences.

There will be explanations as we go along the report and a conclusion summarising the report findings.

-----

# Asian Options:

Asian options, generally speaking, have a payoff that depends on "some" average value of the underlying from the start date to expiry. The average we need to compute depends on the particular type of Asian option. We are going to look at two types: 
 - Asian Fixed Strike Options
 - Asian Floating Strike Options

**Note**: For the first part we will be looking at Arithmetic averaging.

## 1. Asian Fixed Strike Options - Arithmetic Averaging:

The payoffs for the Asian Fixed Strike Option are the following:

For a Call option: $$C_T = max(A - K, 0)$$
Where $A$ represents the average of the stock prices up to maturity and $K$ represents the strike price.

For a Put option: $$P_T = max(K - A, 0)$$
Where $A$ represents the average of the stock prices up to maturity and $K$ represents the strike price.

Here, the strike price, **K** is known and fixed. But the average of the stock prices, **A**, varies with the trajectory of the stock movement.  

**Note**: Next, I will use the code from Professor De Keijzer to generate the stock simulation and also a guide to create my own functions for the options.

**Reference**: Dr Bart De Keijzer(2023). Monte Carlo Simulation for Option Pricing.Monte Carlo Valuation of Asian Fixed-Strike Call Option in Python. Scientific Computing in Finance (last slide).

In [1]:
from numpy import sqrt, exp, cumsum, sum, maximum, mean
from numpy.random import standard_normal
import numpy as np
import pandas as pd

# Parameters
S0 = 100.; T = 1.0; K = 50; r = 0.02; sigma = 0.1; M = 200; dt = T / M; I = 1000000

def inner_value(S):
    # Intrinsic value for a fixed-strike Asian call option
    # This uses a list comprehension (though there is probably a faster, vectorised way possible)
    return np.array([max(V - K,0) for V in mean(S, axis=0)])

# Simulate I paths with M time steps
S = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * 
                          standard_normal((M, I)), axis=0))

# Calculate the Monte Carlo estimator
CT = exp(-r * T) * mean(inner_value(S))
print("Estimated present value is %f" % CT)

Estimated present value is 49.995992


**Note**: The price of Professor De Keijzer's option price is really high due to it's low strike price (K=50). It gives a greater chance of the option being **in-the-money**, increasing it's value.

-----

I will create two functions, for calls and puts by changing the function above.

In [2]:
# Setting the parameters as asked in the coursework:
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.2; M = 252; dt = T / M; I = 100000

# Simulating the numbers again with coursework parameters
simulation1 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

In [3]:
# Function for a call
def present_value_for_asian_fixed_call(S, rate, T=1):
    payoff = np.array([max(V - K,0) for V in mean(S, axis=0)])
    return exp(-r * T) * mean(payoff) # Monte Carlo estimator

In [4]:
# Function for a pull
def present_value_for_asian_fixed_put(S, rate, T=1):
    payoff = np.array([max(K - V,0) for V in mean(S, axis=0)])
    return exp(-r * T) * mean(payoff) # Monte Carlo estimator

Simulating call and put prices with the coursework-specific parameters - i.e initial stock = 100, Strike = 100, Time to expiry = 1 year, Volatility = 20%, and risk-free rate = 5%.

**Note**: The only difference is that I use `K` to denote my strike price throughout the report.

In [5]:
C0 = present_value_for_asian_fixed_call(simulation1, rate=r)
print("The present value of a call option with the given paramaters is:" , round(C0,4))

The present value of a call option with the given paramaters is: 5.74


In [6]:
P0 = present_value_for_asian_fixed_put(simulation1, rate=r)
print("The present value of a put option with the given paramaters is:" ,round(P0,4))

The present value of a put option with the given paramaters is: 3.3646


**Observation**: Running the simulation multiple times I found out that the estimated value of the call tends to range between 5.5 and 5.7 but can be higher or lower. Whereas, the put ranges between 3.3 and 3.5.

## Testing with parameters:

Now, I will change one of the parameters while keeping the others fixed to see the change in the value of the option. I will do this step for interest rates, volatility, and strike price

### Interest rates:

Now, changing the interest rate while keeping the other variables fixed. First, will set the interest rate to 1% and check for both option types and finally increase it to 15% and test it for both option types.

In [7]:
# Simulating new prices with r=0.01
S0 = 100.; T = 1.0; K = 100; r = 0.01; sigma = 0.2; M = 252; dt = T / M; I = 100000
simulation2 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

In [8]:
# Simulating the option prices with r=0.01
C1 = present_value_for_asian_fixed_call(simulation2, rate=r)
P1 = present_value_for_asian_fixed_put(simulation2, rate=r)

print(f"Call price: {round(C1,4)}", f"\nPut price: {round(P1,4)}")

Call price: 4.8809 
Put price: 4.2961


Now with interest rate equal to 15%.

In [9]:
# Calculating new prices with r=0.15
S0 = 100.; T = 1.0; K = 100; r = 0.15; sigma = 0.2; M = 252; dt = T / M; I = 100000
simulation3 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Calculating the option prices with r=0.01
C2 = present_value_for_asian_fixed_call(simulation3, rate=r)
P2 = present_value_for_asian_fixed_put(simulation3, rate=r)

print(f"Call price: {round(C2,4)}" , f"\nPut price: {round(P2,4)}")

Call price: 8.4496 
Put price: 1.6366


In [10]:
# Creating a dataframe
rates_data = {
    "rate = 1%": pd.Series([C1, P1], index=["Call", "Put"]), # The new lower rate
    "rate = 5%": pd.Series([C0, P0], index=["Call", "Put"]), # The default rate at the beginning
    "rate = 15%": pd.Series([C2, P2], index=["Call", "Put"]) # The new upper rate
}

rate_df = pd.DataFrame(rates_data)
rate_df

,rate = 1%,rate = 5%,rate = 15%
Call,4.880946,5.740008,8.449643
Put,4.296107,3.364574,1.636626


In [11]:
# Visualising it better by using transpose
rate_df.T

,Call,Put
rate = 1%,4.880946,4.296107
rate = 5%,5.740008,3.364574
rate = 15%,8.449643,1.636626


**Explanation**: As we can see, increasing the interest rate results in an increase in the call option prices but a decrease in the put option prices.

Although it is not part of the report, it would be interesting to see how this would change if we compare this specific setting but now instead of having 1 year for time to expiry we have 2 or 4.

### Volatility:

Now, will test with interest rates being 5% and then 40%. After we will compare with the initial 20% and see any patterns.

In [12]:
# Simulating new prices with sigma = 0.05
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.05; M = 252; dt = T / M; I = 100000
simulation4 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Calculating corresponding call and put prices with sigma = 0.05
C3 = present_value_for_asian_fixed_call(simulation4, rate=r)
P3 = present_value_for_asian_fixed_put(simulation4, rate=r)


# Simulating new prices with sigma = 0.40
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.40; M = 252; dt = T / M; I = 100000
simulation5 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Calculating corresponding call and put prices with sigma = 0.40
C4 = present_value_for_asian_fixed_call(simulation5,rate=r)
P4 = present_value_for_asian_fixed_put(simulation5, rate=r)


In [13]:
# Generating data frame for comparison

volatility_data = {
    "volatility = 5%": pd.Series([C3, P3], index=["Call", "Put"]), # The new lower rate
    "volatility = 20%": pd.Series([C0, P0], index=["Call", "Put"]), # The default rate at the beginning
    "volatility = 40%": pd.Series([C4, P4], index=["Call", "Put"]) # The new upper rate
}

# Visualising it
volatility_df = pd.DataFrame(volatility_data)
volatility_df

,volatility = 5%,volatility = 20%,volatility = 40%
Call,2.711012,5.740008,10.344251
Put,0.301134,3.364574,7.695758


In [14]:
# Visualise the table better
volatility_df.T

,Call,Put
volatility = 5%,2.711012,0.301134
volatility = 20%,5.740008,3.364574
volatility = 40%,10.344251,7.695758


**Result**: As volatility increases, the prices of both call and put options tend to rise. This is likely because with higher volatility we have higher price fluctuations, this will increases the probability of the option being **in-the-money** (generating a payoff greater than 0 respective to the type of option we are dealing with). Ultimatately, this result in the increase in the price of the options.

### Strike Price:

Finally, let's check the impact of changing the strike price to 50 and then 150 - The code will be summarised here.

In [15]:
# Setting the strike price to 50
S0 = 100.; T = 1.0; K = 50; r = 0.05; sigma = 0.2; M = 252; dt = T / M; I = 100000

# Simulating the prices again with strike = 50
simulation6 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Calculating option prices with K=50
C5 = present_value_for_asian_fixed_call(simulation6, rate=r)
P5 = present_value_for_asian_fixed_put(simulation6, rate=r)


# Setting the strike price to 150
S0 = 100.; T = 1.0; K = 150; r = 0.05; sigma = 0.2; M = 252; dt = T / M; I = 100000

# Simulating the prices again with strike = 150
simulation7 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Calculating option prices with K=150
C6 = present_value_for_asian_fixed_call(simulation7, rate=r)
P6 = present_value_for_asian_fixed_put(simulation7,rate=r)


# generating data frame for strike prices
strike_data = {
    "strike = 50": pd.Series([C5, P5], index=["Call", "Put"]), # The new lower value
    "strike = 100": pd.Series([C0, P0], index=["Call", "Put"]), # The default value
    "strike = 150": pd.Series([C6, P6], index=["Call", "Put"]) # The new upper value
}

strike_df = pd.DataFrame(strike_data)
strike_df.T

,Call,Put
strike = 50,49.990641,0.000000
strike = 100,5.740008,3.364574
strike = 150,0.003337,45.171121


**Result**: As expected, calls and puts behave in the opposite way to the change in strike price. Increasing the strike price results in a reduction in a call option and increase in a put option. This is because the payoff of a call is: `max(A - K, 0)`. So, higher the strike, the more likely the stock will be **out-of-the-money**, resulting a lower price. For a out the payoff is: `max(K - A, 0)`, this has the opposite effect, resulting in higher prices of the put option as the strike increases because it increases the possibility to be **in-the-money**.

**Extra**: I was not sure if we were required to create a function from scratch to price the options. So I created one below which incorporates both functions from earlier in which we are required to input the type of option we would like to price between a call or put. I am not going to re-do all the previous analysis again, but show that they generate similar results below.

In [16]:
# Asian option with fixed strike function
def asian_fixed_strike_price(stock_simulation, option_type, K=100, rate=0.05, T=1):
    option_type = str(option_type) # Type of option to price
    discounting = exp(-r * T) # Creating discounting factor
    
    if option_type.lower() in ["call", "call option", "call_option"]:
        value = np.array([max(V - K, 0) for V in np.mean(stock_simulation, axis=0)])
        discounted_option_price_call = discounting * mean(value) # Monte Carlo estimator
        return discounted_option_price_call

    elif option_type.lower() in ["put", "put option", "put_option"]:
        value = np.array([max(K - V, 0) for V in np.mean(stock_simulation, axis=0)])
        discounted_option_price_put = discounting * mean(value) # Monte Carlo estimator
        return discounted_option_price_put
    
    else:
        return "The type of option was not clear."

In [17]:
# Creating two (first two) option price to comapre with the ones computes with the functions separately
# Creating the third to show if we give wrong input name
c_compare11 = asian_fixed_strike_price(simulation1, "call", rate=0.05)
p_compare11 = asian_fixed_strike_price(simulation1, "put", rate=0.05)
wrong_input = asian_fixed_strike_price(simulation1, "pat", rate=0.05)

# Printing the output
print(f"Call price: {c_compare11}", f"\nPut price:  {p_compare11}", f"\nOutput if give the wrong input, in this case we input 'pat': {wrong_input}")

Call price: 5.740007917265428 
Put price:  3.364573967141617 
Output if give the wrong input, in this case we input 'pat': The type of option was not clear.


As we can see, the prices are fairly similar to the prices generated at the beginning, `C0` and `P0`, for the first two options. This suggest the function should work correctly.

------

## 2. Asian Floating Strike Options - Arithmetic Averaging:

The payoffs for the Asian Floating Strike Option are the following:

For a Call option: $$C_T = max(S(T) - A, 0)$$
Where $A$ represents the strike price and it is calculated from the average of the stock prices up to maturity and $S(T)$ represents the stock price(s) at maturity.

For a Put option: $$P_T = max(A - S(T), 0)$$
Where $A$ represents the strike price and it is calculated from the average of the stock prices up to maturity and $S(T)$ represents the stock price(s) at maturity.

Here, we have a strike price that will change based on the fluctuations of the stock prices. Both the strike prices and final prices of the underlying will be known only at maturity.

Let's use the codes for the fixed strike Asian Options (both call and put) to create the function for the floating strike Asian options

In [18]:
def present_value_for_asian_floating_call(S, rate, T=1):
    prices_at_maturity = S[-1] # Prices at maturity
    
    floating_strike = np.mean(S, axis=0) # Average of the stock prices 
    payoff = np.maximum(prices_at_maturity - floating_strike, 0) # Here we need to use np.maximum, to compute an element-wise maximum, which we then use to compute the mean (as our Monte-Carlo estimator)
    return exp(-r * T) * mean(payoff) # Returning the present value

def present_value_for_asian_floating_put(S, rate, T=1):
    prices_at_maturity = S[-1] # Prices at maturity
    
    floating_strike = np.mean(S, axis=0) # Average of the stock prices 
    payoff = np.maximum(floating_strike - prices_at_maturity, 0) # Similar as above
    return exp(-r * T) * mean(payoff) # Returning the present value

Now, let's test the functions using the stock prices generated earlier, `simulation1`, and let's compare their prices against their corresponding fixed call and put Asian options calculated earlier.

In [19]:
floating_call1 = present_value_for_asian_floating_call(simulation1, rate=0.05)
print("The estimated present value of the Asian floating call is: ", floating_call1)

The estimated present value of the Asian floating call is:  5.832515746649978


In [20]:
floating_put1 = present_value_for_asian_floating_put(simulation1, rate=0.05)
print("The estimated present value of the Asian floating put is: ", floating_put1)

The estimated present value of the Asian floating put is:  3.4109874387756767


### Comparing Asian Fixed Strike and Asian Floating Strike:

Before comparing the prices of the call and put Fixed Strike Asian options against the call and put prices of Floating Strike Asian options over the same stock simulation, `simulation1`, let's create a function a "compact" function that asks us what type of option we would like to price. This is similar as done before for Fixed Strike Asian options.

In [21]:
# Creating a function for both put and call floating strike asian option
def present_value_for_asian_floating(S, option_type, rate, T=1):
    option_type = str(option_type) # Deciding the type of the option
    prices_at_maturity = S[-1] # Prices at maturity
    floating_strike = np.mean(S, axis=0) # Average of the stock prices
    discounting = exp(-r * T) # Creating discounting factor
    
    if option_type.lower() in ["call", "CALL", "call option"]:
        payoff = np.maximum(prices_at_maturity - np.mean(S, axis=0), 0) # Here we need to use np.maximum want to compute an element-wise maximum, which we then use to compute the mean (as our Monte-Carlo estimator).
        return discounting * mean(payoff) # Returning the present value
    
    elif option_type.lower().strip() in ["put", "PUT", "put option"]:
        payoff = np.maximum(np.mean(S, axis=0) - prices_at_maturity, 0) # Here we need to use np.maximum again - same as above
        return discounting * mean(payoff) # Returning the present value
    
    else:
        return "The type of option was not clear. Please, try again"

In [22]:
# Just using the new function and ovewriting the results into those variables
floating_call1 = present_value_for_asian_floating(simulation1, "call", rate=0.05)
floating_put1 = present_value_for_asian_floating(simulation1, "PUT", rate=0.05)

In [23]:
# Putting the prices together into a data frame
fixed_vs_floating = {
    "Fixed Strike": pd.Series([C0, P0], index=["Asian Call Price", "Asian Put Price"]),
    "Floating Strike": pd.Series([floating_call1, floating_put1], index=["Asian Call Price", "Asian Put Price"])
}

# Constructing the DataFrame from the dictionary
result_df = pd.DataFrame(fixed_vs_floating)
result_df

,Fixed Strike,Floating Strike
Asian Call Price,5.740008,5.832516
Asian Put Price,3.364574,3.410987


**Explanation/Description**: 
As shown in the table above, the floating strike options prices are slightly higher than their fixed strike "counterparts". The explanation for this would be that the floating types are more likely to be "**in the money**". This is because,unlikely the fixed strike where the strike remains constant over the life of the option, the strike (in the flaoting) changes based on how the market moves.

**Interesting Observation**: However, it is important to know that sometimes, depending on how numbers are generated, that the fixed prices could be higher than the floating ones. I noticed this when running the simulation multiple times but most of the time is aligns with the explanation above.

----

### Extra - Parameter change comparison: Fixed Asian Option and Floating Asian Option

I am not sure if this is required but I will want to include it. In this section I am comparing how (if any) changing the interest rate and volatility affects the option prices for a call and put for the floating type. The main goal is to compare if there are differences or similarities between fixed and floating.

The comparison will use similar settings. In this sense, the same change in interest rates and volatility. Also, there will be no example of changing the strike as we are dealing with floating strike options.

Feel free to skip to the next part `3. Asian Fixed/Floating Strike Options - Geometric Averaging:` and `Supershare options` if you'd like. 

Also, we will be only focusing on the tables without putting much emphasis on the code as it is the same as before but with some small changes for generating numbers and renaming variables. The variables' names can be confusing as I ran out of names.

Let's first do it for the **volatility**.

In [24]:
# Simulating new prices with sigma = 0.05
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.05; M = 252; dt = T / M; I = 100000
simulationggg1 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Simulating corresponding call and put prices with sigma = 0.05
xxx1 = present_value_for_asian_floating(simulationggg1, "call",rate=r)
yyy1 = present_value_for_asian_floating(simulationggg1, "put",rate=r)

# Simulating new prices with sigma = 0.40
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.40; M = 252; dt = T / M; I = 100000
simulationjjj1 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

# Simulating corresponding call and put prices with sigma = 0.40
ppp1 = present_value_for_asian_floating(simulationjjj1, "call",rate=r)
qqq1 = present_value_for_asian_floating(simulationjjj1, "put",rate=r)

# Generating data frame for comparison
gf_data = {
    "volatility = 5%": pd.Series([xxx1, yyy1], index=["Call", "Put"]),
    "volatility = 20%": pd.Series([C0, P0], index=["Call", "Put"]),
    "volatility = 40%": pd.Series([ppp1, qqq1], index=["Call", "Put"])
}

# Visualising it
gf_df = pd.DataFrame(gf_data)
gf_df.T

,Call,Put
volatility = 5%,2.754452,0.302781
volatility = 20%,5.740008,3.364574
volatility = 40%,10.265219,7.850716


**Explanation**: As we can see from the table above, it behaves in the same pattern as the fixed strike Asian option.

Now, with the **interest rate**.

In [25]:
# Simulating new prices with r=0.01
S0 = 100.; T = 1.0; K = 100; r = 0.01; sigma = 0.2; M = 252; dt = T / M; I = 100000
simulationggg2 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

xxx2 = present_value_for_asian_floating(simulationggg2, "call", rate=r)
yyy2 = present_value_for_asian_floating(simulationggg2, "put", rate=r)

# Simulating new prices with r=0.15
S0 = 100.; T = 1.0; K = 100; r = 0.15; sigma = 0.2; M = 252; dt = T / M; I = 100000
simulationjjj2 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

ppp2 = present_value_for_asian_floating(simulationjjj2, "call",  rate=r)
qqq2 = present_value_for_asian_floating(simulationjjj2, "put", rate=r)

kf_data = {
    "rate = 1%": pd.Series([xxx2, yyy2], index=["Call", "Put"]),
    "rate = 5%": pd.Series([C0, P0], index=["Call", "Put"]),
    "rate = 15%": pd.Series([ppp2, qqq2], index=["Call", "Put"])
}

kf_df = pd.DataFrame(kf_data)
kf_df.T

,Call,Put
rate = 1%,4.823190,4.316382
rate = 5%,5.740008,3.364574
rate = 15%,8.816388,1.701262


**Explanation**: Again, the same pattern as with the fixed Asian options

-----

## 3. Asian Fixed/Floating Strike Options - Geometric Averaging:

So far we used the Arithmetic averaging which can be represented using the following formula:
$$ A = \frac{1}{n} \sum_{i=1}^{n} S({t_i}) $$

Now, we are going to test the Geometric averaging, which is represented by:

$$ {A_G} = \left( \prod_{i=1}^{n} S({t_i}) \right)^{\frac{1}{n}} $$

The Geometric average can also be represente in the following after some manipulation: 

$$ {A_G} = e^{\frac{1}{n} \sum_{i=1}^{n} \log(S({t_i}))} $$

We will use the last form in my function below.

In [26]:
def present_value_for_geometric_asian(S, option_type, strike_type, rate, time=1):
    option_type = str(option_type) # Type of option we want (put or call)
    strike_type = str(strike_type) # Type of strike we want (fixed or flaoting)
    prices_at_maturity = S[-1] # Prices at maturity
    geometric_strike = np.exp(np.mean(np.log(S), axis=0)) # Geometric average of the stock prices
    discounting = exp(-r * T) # Creating discounting factor
    
    # The rest is the same as before, only difference we apply the geometric mean
    if strike_type.title() == "Fixed": # If fixed
        if option_type.lower() in ["call", "call option", "call_option"]: # For calls
            value = np.array([max(V - K, 0) for V in geometric_strike])
            discounted_option_price_call = discounting * np.mean(value) # Monte Carlo averaging
            return discounted_option_price_call
        
        elif option_type.lower() in ["put", "put option", "put_option"]: # For puts
            value = np.array([max(K - V, 0) for V in geometric_strike])
            discounted_option_price_put = discounting * np.mean(value) # Monte Carlo averaging
            return discounted_option_price_put
        
        else:
            return "The type of option was not clear."
    
    elif strike_type.title() == "Floating": # If floating
        if option_type.lower() in ["call", "CALL", "call option"]: # For calls
            payoff = np.maximum(prices_at_maturity - geometric_strike, 0)
            return discounting * np.mean(payoff) # Monte Carlo averaging
        
        elif option_type.lower() in ["put", "PUT", "put option"]: # For puts
            payoff = np.maximum(geometric_strike - prices_at_maturity, 0)
            return discounting * np.mean(payoff) # Monte Carlo averaging
        
        else:
            return "The type of option was not clear. Please try again."

In [27]:
# Simulating combination of call and put options with fixed and floating strikes. They have the same paramaters as the one in the worksheet
geom_fixed_call = present_value_for_geometric_asian(simulation1, "CALL", "fixed", rate=0.05)
geom_fixed_put = present_value_for_geometric_asian(simulation1, "PUT", "fixed", rate=0.05)
geom_floating_call = present_value_for_geometric_asian(simulation1, "call", "floating", rate=0.05)
geom_floating_put = present_value_for_geometric_asian(simulation1, "PUT", "floating", rate=0.05)

In [28]:
# Recalling the simulations again
floating_call1 = present_value_for_asian_floating(simulation1, "call", rate=0.05)
floating_put1 = present_value_for_asian_floating(simulation1, "PUT", rate=0.05)
fixed_call1 = present_value_for_asian_fixed_call(simulation1, rate=0.05)
fixed_put1 = present_value_for_asian_fixed_put(simulation1, rate=0.05)

Next, I will compare arithmetic and geometric averaging by combining the results into a data frame. These values have been simulated from the same stock simulation, `simulation1`, same interest rate and time frame, `rate=0.05` and `T=1`. These are the initial values given in the coursework and I thought they would be ideal to use for comparison.

In [29]:
# Creating the data frame for comparison
fixed_vs_floating = {
    "Arithmetic Averaging": pd.Series([fixed_call1, fixed_put1, floating_call1, floating_put1],
                                      index=["Fixed Call", "Fixed Put", "Floating Call", "Floating Put"]),
    "Geometric Averaging": pd.Series([geom_fixed_call, geom_fixed_put,
                                      geom_floating_call, geom_floating_put],
                                      index=["Fixed Call", "Fixed Put", "Floating Call", "Floating Put"])
}

fixed_vs_floating_df = pd.DataFrame(fixed_vs_floating)
fixed_vs_floating_df

,Arithmetic Averaging,Geometric Averaging
Fixed Call,5.193774,4.998712
Fixed Put,3.044392,3.151462
Floating Call,5.277478,5.465766
Floating Put,3.086389,2.972545


**Results/Explanations**: From running the code (stock simulations) multiple times, I can see that usually, the geometric averaging yields a lower price in most of the cases. However, there are some instances where this "condition" is violated. So, it can happen to have the **Floating Call** for for the **Geometric Averaging** being larger than its **Arithmetic Averaging** counterpart. 

**Note**: If you see "strange" results, please re-run the code from `3. Asian Fixed/Floating Strike Options - Geometric Averaging` few times to re-generate the stock prices. This is just the last 4/5 "blocks" of code.

-----

## Supershare Options:

Let's recall a "standard" binary option first. A binary option is an option in which the payoff is either 1 or 0 depending if the option is **in-the-money** or **out-the-money**.

A **Supershare** option is a type of binary option in which the payoff is given by dividing the price underlying at expiry by the lower boundary also called a lower strike. The option will only yield a positive payoff if the price of the underlying is within the lower and upper strikes at maturity. Otherwise, it's zero.

In [30]:
# Creating  a function of the supershare option
def super_share_option_price(S, lower_strike, upper_strike, r, T=1):
    S_t_array = np.array(S[-1]) # Prices at expiry
    discounted_payoff_lst = [] # Appending the discounted payoff here for each realisation 
    
    for final_price in S_t_array:
        if final_price >= upper_strike or final_price < lower_strike:
            discounted_payoff_lst.append(0) # If outside the in-the-money zone
        else:
            discounted_payoff_lst.append((final_price/lower_strike) * exp(-r * T)) # If inside the in-the-money zone
    return mean(discounted_payoff_lst)

Let's price a supershare option with the "default" interest rate of 5% and time to maturity of 1 year, but set the lower strike to 90 and the upper strike to 110.

In [31]:
S0 = 100.; T = 1.0; K = 100; r = 0.05; sigma = 0.2; M = 252; dt = T / M; I = 100000

# Simulating the numbers again with coursework parameters
simulation1 = S0 * exp(cumsum((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * standard_normal((M, I)), axis=0))

In [32]:
super1 = super_share_option_price(simulation1, 90, 110, r)
print(f"Price of a supershare option with lower strike = 90, upper strike = 110, interest rate = 5%:\
\n{round(super1,5)}")

Price of a supershare option with lower strike = 90, upper strike = 110, interest rate = 5%:
0.4003


**Scenario 1**: Now, let's change the upper strike while keeping everything else the same to assess the change in the price of the super option:

In [33]:
super2 = super_share_option_price(simulation1, 90, 120, r)
print(f"Price of a supershare option with lower strike = 90, upper strike = 120, interest rate = 5%:\
\n{round(super2,5)}")

Price of a supershare option with lower strike = 90, upper strike = 120, interest rate = 5%:
0.5787


Let's have a series of supershare options but each time we make the gap between the lower and upper strike larger. 

In [34]:
super3 = super_share_option_price(simulation1, 90, 170, r)
print(f"Price of a supershare option with lower strike = 90, upper strike = 170, interest rate = 5%:\
\n{round(super3,5)}")

Price of a supershare option with lower strike = 90, upper strike = 170, interest rate = 5%:
0.88494


In [35]:
super4 = super_share_option_price(simulation1, 90, 900, r)
print(f"Price of a supershare option with lower strike = 90, upper strike = 900, interest rate = 5%:\
\n{round(super4,5)}")

Price of a supershare option with lower strike = 90, upper strike = 900, interest rate = 5%:
0.89679


In [36]:
# Creating a data frame for comparison
strike_data1 = {
    "Strikes: lower = 90 , upper = 110": pd.Series([super1], index=["Super option price"]),
    "Strikes: lower = 90 , upper = 120": pd.Series([super2], index=["Super option price"]),
    "Strikes: lower = 90 , upper = 170": pd.Series([super3], index=["Super option price"]),
    "Strikes: lower = 90 , upper = 900": pd.Series([super4], index=["Super option price"]),
}

strike1_df = pd.DataFrame(strike_data1)
strike1_df.T

,Super option price
"Strikes: lower = 90 , upper = 110",0.400305
"Strikes: lower = 90 , upper = 120",0.578700
"Strikes: lower = 90 , upper = 170",0.884939
"Strikes: lower = 90 , upper = 900",0.896790


**Explanation/Result**: As we increase the upper strike while keeping the lower strike fixed the price we can see the price of the option increasing which makes sense as there is more probability of the payoff being **in-the-money** with a greater range. But we can also see that the price does not increase by a significant amount. I believe this is because the lower strike is quite close to the initial price which lowers the chances of the payoff being **in-the-money** than if it was, say 25. This is not included in the report but this is what I would normally expect.

**Scenario 2**: Now, let's decrease the lower strike, while keeping the upper strike fixed at 110.

In [37]:
super5 = super_share_option_price(simulation1, 80, 110, r)
print(f"Price of a supershare option with lower strike = 80, upper strike = 110, interest rate = 5%:\
\n{round(super5,5)}")

Price of a supershare option with lower strike = 80, upper strike = 110, interest rate = 5%:
0.6


In [38]:
super6 = super_share_option_price(simulation1, 30, 110, r)
print(f"Price of a supershare option with lower strike = 30, upper strike = 110, interest rate = 5%:\
\n{round(super6,5)}")

Price of a supershare option with lower strike = 30, upper strike = 110, interest rate = 5%:
1.84062


In [39]:
super7 = super_share_option_price(simulation1, 1, 110, r)
print(f"Price of a supershare option with lower strike = 1, upper strike = 110, interest rate = 5%:\
\n{round(super7,5)}")

Price of a supershare option with lower strike = 1, upper strike = 110, interest rate = 5%:
55.21871


In [40]:
# Creating a data frame for comparison
strike_data2 = {
    "Strikes: lower = 90 , upper = 110": pd.Series([super1], index=["Super option price"]),
    "Strikes: lower = 80 , upper = 110": pd.Series([super5], index=["Super option price"]),
    "Strikes: lower = 30 , upper = 110": pd.Series([super6], index=["Super option price"]),
    "Strikes: lower = 1 , upper = 110": pd.Series([super7], index=["Super option price"]),
}

strike2_df = pd.DataFrame(strike_data2)
strike2_df.T

,Super option price
"Strikes: lower = 90 , upper = 110",0.400305
"Strikes: lower = 80 , upper = 110",0.599998
"Strikes: lower = 30 , upper = 110",1.840624
"Strikes: lower = 1 , upper = 110",55.218710


**Explanation/Result**: The results are as expected, decreasing the lower strike while keeping the upper strike fixed increases the option price for the same reason as above. But what I can see is that the option price increases much more than the previous example. This may suggest that the prices at maturity tend to go down more actually increasing in value making them more likely to "break" the lower barrier than the upper one.

**Scenario 3**: So far we made the "gap" between the strikes larger (one at a time). Now, let's make them smaller simultaneously. This time, I will summarise them directly in a table.

In [41]:
# Pricing and putting into a table
super8 = super_share_option_price(simulation1, 92, 107, r)
super9 = super_share_option_price(simulation1, 105, 105, r)
super10 = super_share_option_price(simulation1, 106, 105, r)

# Creating a data frame
strike_data3 = {
    "Strikes: lower = 90 , upper = 110": pd.Series([super1], index=["Supershare option price"]),
    "Strikes: lower = 92 , upper = 105": pd.Series([super8], index=["Supershare option price"]),
    "Strikes: lower = 105 , upper = 110": pd.Series([super9], index=["Supershare option price"]),
    "Strikes: lower = 106 , upper = 105": pd.Series([super10], index=["Supershare option price"]),
}

strike3_df = pd.DataFrame(strike_data3)
strike3_df.T

,Supershare option price
"Strikes: lower = 90 , upper = 110",0.400305
"Strikes: lower = 92 , upper = 105",0.298108
"Strikes: lower = 105 , upper = 110",0.000000
"Strikes: lower = 106 , upper = 105",0.000000


**Explanation/Result**: Simple and expected results, the closer the strikes are to the initial price of the underlying, the more likely it will be **out-the-money** which means lower option prices.

**Scenario 4 - My question**: What if the stock starts below both strike prices - i.e below the lower strike? Will it end up in the money or out the money? What about when the stock starts above both the strike prices - i.e above the upper strike?

**Note**: Recall that in the simulations, we set the initial stock value as 100 , `S0 = 100`.

In [42]:
super11 = super_share_option_price(simulation1, 85, 95, r)
super12 = super_share_option_price(simulation1, 110, 115, r)

strike_data4 = {
    "Strikes: lower = 90 , upper = 110": pd.Series([super1], index=["Supershare option price"]),
    "Strikes: lower = 85 , upper = 95": pd.Series([super11], index=["Supershare option price"]),
    "Strikes: lower = 110 , upper = 115": pd.Series([super12], index=["Supershare option price"]),
}

strike4_df = pd.DataFrame(strike_data4)
strike4_df.T

,Supershare option price
"Strikes: lower = 90 , upper = 110",0.400305
"Strikes: lower = 85 , upper = 95",0.176131
"Strikes: lower = 110 , upper = 115",0.077350


**Explanation/Result**: The first row of the table is just a "base" of the option price using the "default" parameters. The second row shows that if the stock price starts above both strikes it tends to generate a higher price of the option than if it started below both strikes. Recall again that we simulated the stocks with an initial price of 100. 

# Conclusion:

In conclusion,  both Fixed Strike Asian and Floating Strike Asian options can exhibit similar behaviors to the same changes in their parameters. Usually, Floating Strike Asian Options tend to have their prices slightly higher than their Fixed Strike counterparts. When using the Geometric averaging their prices are generally lower. Finally, the value of the Supershare option changes based on the initial stock price to generate the simulations but also the lower and upper strike prices.

**References**:

Reference 1: Dr Bart de Keijzer(2023). Monte Carlo Simulation for Option Pricing.Monte Carlo Valuation of Asian Fixed-Strike Call Option in Python. Scientific Computing in Finance (last slide).

-----